In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB

import numpy as np
from sklearn.decomposition import PCA

import types
import copy





Using TensorFlow backend.


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [5]:
# db_lgbm = DFDB('../trial/lgbm.pkl', auto_commit=False)
# df_trial_lgbm = db_lgbm.select()
# df_trial_lgbm['kfold'] = df_trial_lgbm['param'].apply(lambda x: x['kfold'])
# df_trial_lgbm['kfold-type'] = df_trial_lgbm['param'].apply(lambda x: x['kfold']['type'])

In [6]:
# df_trial_lgbm[(df_trial_lgbm['kfold-type']=='group')&(df_trial_lgbm['mae_diff']<.2)].sort_values(by=['val_mae'])[['datetime','nfeatures','kfold-type','train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head()

,datetime,nfeatures,kfold-type,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
2498,2019-05-23 23:32:01.229662,14,group,1.882727,0.010383,2.058523,0.488644,0.175796
2546,2019-05-24 00:04:53.793038,14,group,1.872995,0.010356,2.060622,0.477251,0.187627
2515,2019-05-23 23:41:56.647154,14,group,1.867831,0.010190,2.060701,0.487186,0.192870
2473,2019-05-23 23:16:18.127971,14,group,1.889906,0.010449,2.060748,0.482239,0.170842
2367,2019-05-23 21:45:02.439202,14,group,1.891329,0.010296,2.061111,0.478845,0.169782


In [5]:
df_spec_train = pd.read_pickle('../feats/spec_features.pkl')
df_spec_test = pd.read_pickle('../feats/spec_features_test.pkl')

In [6]:
df_train = pd.merge(df_train, df_spec_train, on='index')
df_test = pd.merge(df_test, df_spec_test, on='index')

In [7]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [8]:
len(tsfresh_columns)

9731

In [14]:
db = DFDB('../trial/lgbm4.pkl', auto_commit=False)

In [12]:
# group = EP.bubble_merge(df_train, columns=tsfresh_columns, base_correlation_coefficient=.9, coverage_rate=.9)

In [69]:
param822 = {'columns': tsfresh_columns,
 'kfold': {'n_splits': 4,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler', 'init':{}},
 'algorithm': {'cls': 'lgb.LGBMRegressor',
  'init': {'learning_rate': 0.20562566891912665,
   'feature_fraction': 0.6442827431025538,
   'bagging_fraction': 0.802261802916953,
   'min_data_in_leaf': 704,
   'lambda_l1': 65.43072344708864,
   'lambda_l2': 62.4903783326351,
   'max_bin': 96,
   'num_leaves': 4,
   'random_state': 5943,
   'n_jobs': 16},
  'fit': {}},
 }

In [28]:
mytrial =[]
df_his, df_feature_importances, df_valid_pred, df_test_pred = EP.process(df_train, param822, df_test=df_test, trial=mytrial, is_output_feature_importance=True, remark='init_try')

In [30]:
df_trial = pd.DataFrame(mytrial)

In [33]:
df_trial[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
0,2019-05-30 06:03:08.407859,9731,1.833261,0.002514,2.206499,0.018641,0.373238


In [31]:
df_feature_importances = df_trial['df_feature_importances'].values[0]
top200 = EP.evaluate(df_feature_importances)[:200]

In [23]:
top200 =['spkt_welch_densitycoeff_2',
 'spkt_welch_density__coeff_3',
 'q25_roll_std_100',
 '3th_peak_freq',
 'min_roll_std_100',
 'q05_roll_std_100',
 'iqr_6',
 'abs_max_8',
 'mfcc_5_abs_mean',
 "number_peaks{'n': 10}",
 'ave10_7',
 'mfcc_13_mean',
 "autocorrelation{'lag': 5}",
 'mfcc_accelerate_8_variance',
 'abs_max_7',
 'mfcc_accelerate_1_kurtosis',
 'q05_roll_std_1000',
 'spkt_welch_density__coeff_42',
 'max_to_min_diff_5',
 'mfcc_13_quantile25',
 'fft_coefficientcoeff_80__attr_"imag"',
 'abs_q25_5',
 'mfcc_5_mean',
 'median__roll_std',
 '5000skewness_max_',
 'fft_coefficientcoeff_6__attr_"abs"',
 'partial_autocorrelationlag_5',
 'abs_min_8',
 'spkt_welch_density__coeff_28',
 'ar_coefficientk_10__coeff_3',
 'abs_q75_7',
 'mfcc_accelerate_15_min',
 'abs_max_4',
 'mfcc_10_quantile25',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'spkt_welch_density__coeff_79',
 "value_count{'value': 1}",
 '5000peak_peak_amp_max_',
 'min__roll_std',
 'mfcc_4_rolling_std_mean',
 'q01_roll_std_1000',
 'mfcc_3_abs_q75',
 'mfcc_5_quantile95',
 'fft_coefficientcoeff_16__attr_"imag"',
 'abs_q01_4',
 'mfcc_6_quantile05',
 'agg_autocorrelationf_agg_"mean"__maxlag_40',
 '5000quantile25skewness_',
 'mfcc_0_abs_q25',
 'mfcc_delta_3_quantile01',
 'spkt_welch_density__coeff_99',
 'mfcc_9_mean',
 'mfcc_12_mean',
 'q75_roll_mean_10',
 'max_to_min',
 'mfcc_3_abs_mean',
 'kurt_7',
 'mfcc_14_quantile01',
 'mfcc_10_mean',
 'mfcc_delta_13_min',
 'mfcc_delta_13_abs_max',
 '5000no_zero_crossing_mean_',
 'abs_q75_6',
 'mfcc_12_quantile99',
 'mfcc_accelerate_6_skewness',
 'mfcc_5_max',
 '5000smoothness_entropy_',
 '5000median_skewness_',
 'mfcc_accelerate_8_min',
 'abs_min_3',
 '5000quantile75mean_',
 "number_crossing_m{'m': 1}",
 '5000smoothness_std_',
 'mfcc_9_abs_q25',
 'flac3_1_quantile99',
 'mfcc_5_skewness',
 'fft_coefficientcoeff_56__attr_"angle"',
 'fft_coefficientcoeff_70__attr_"abs"',
 'mfcc_10_abs_q75',
 'fft_coefficientcoeff_24__attr_"angle"',
 'med_7',
 'spkt_welch_density__coeff_73',
 'abs_q99_8',
 'ave10_6',
 'spkt_welch_density__coeff_38',
 'skew_1',
 'mfcc_delta_3_abs_q95',
 "change_quantiles{'ql': 0.6, 'qh': 1.0, 'isabs': False, 'f_agg': 'mean'}",
 'max_to_min_5',
 'mfcc_delta_4_quantile75',
 'abs_max_1',
 '5000crest_factor_quantile75',
 'partial_autocorrelationlag_1',
 'mfcc_delta_11_quantile01',
 'mfcc_accelerate_7_quantile01',
 'mfcc_8_rolling_std_mean',
 'q95_roll_mean_100',
 'mfcc_delta_6_rolling_std_mean',
 'flac3_0_min',
 'fft_coefficientcoeff_36__attr_"abs"',
 'iqr_8',
 'max_9',
 '5000smoothness_quantile05',
 'mfcc_1_kurtosis',
 'mfcc_7_abs_q95',
 'spkt_welch_density__coeff_66',
 'spkt_welch_density__coeff_64',
 'mfcc_3_abs_q95',
 '5000skewness_mean_',
 'mfcc_10_abs_mean',
 '5000quantile99quantile01',
 'mean_change_rate',
 'mfcc_accelerate_5_rolling_std_mean',
 'mfcc_accelerate_5_quantile05',
 '5000rms_median_',
 'flac3_1_abs_mean',
 '5000rms_quantile05',
 '5000quantile75quantile25',
 'mfcc_accelerate_12_min',
 "value_count{'value': -1}",
 'mfcc_12_abs_max',
 'abs_min_5',
 'mfcc_5_abs_q99',
 'mfcc_delta_5_quantile01',
 'mfcc_12_quantile75',
 'fft_coefficientcoeff_56__attr_"imag"',
 'spkt_welch_densitycoeff_5',
 'mfcc_0_abs_q05',
 'mfcc_13_quantile75',
 'mfcc_delta_5_min',
 'mfcc_5_quantile99',
 'fft_coefficientcoeff_8__attr_"angle"',
 'spkt_welch_density__coeff_30',
 'mfcc_accelerate_4_max',
 'mfcc_14_median',
 "change_quantiles{'ql': 0.2, 'qh': 0.4, 'isabs': False, 'f_agg': 'var'}",
 'mfcc_5_median',
 'mfcc_accelerate_5_abs_q75',
 'spkt_welch_density__coeff_59',
 'mfcc_accelerate_5_kurtosis',
 'mfcc_delta_9_quantile99',
 'mfcc_5_quantile25',
 'spkt_welch_density__coeff_113',
 'mfcc_2_quantile05',
 'spkt_welch_density__coeff_58',
 'mfcc_5_abs_q75',
 'spkt_welch_density__coeff_22',
 'spkt_welch_density__coeff_115',
 'spkt_welch_density__coeff_4',
 'mfcc_5_abs_q05',
 'spkt_welch_density__coeff_25',
 'mfcc_5_abs_max',
 'spkt_welch_density__coeff_27',
 'mfcc_4_quantile75',
 'mfcc_3_rolling_std_mean',
 'mfcc_4_median',
 'mfcc_0_quantile01',
 '5000quantile75rssq_',
 'fft_coefficientcoeff_24__attr_"imag"',
 'mfcc_delta_12_kurtosis',
 '5000rms_quantile25',
 'mfcc_13_quantile05',
 'fft_coefficientcoeff_8__attr_"imag"',
 'mfcc_delta_11_rolling_std_mean',
 'mfcc_delta_12_quantile99',
 'mfcc_accelerate_9_abs_q25',
 'mfcc_11_abs_mean',
 'mfcc_delta_15_abs_max',
 'abs_max_2',
 'mfcc_11_abs_std',
 'abs_max_roll_mean_1000',
 'peak_to_average_power_ratio__roll_mean',
 'mfcc_13_median',
 'fft_coefficientcoeff_62__attr_"abs"',
 'mfcc_11_quantile75',
 "quantile{'q': 0.8}",
 'mfcc_delta_3_rolling_std_mean',
 'abs_q99_7',
 'mfcc_delta_12_max',
 'mfcc_12_min',
 'mfcc_10_quantile99',
 'mfcc_14_abs_q99',
 'mfcc_14_quantile05',
 'mfcc_14_quantile25',
 'kurt_1',
 'mfcc_2_median',
 'q01_2',
 'mfcc_2_abs_q25',
 'kurt_8',
 'mfcc_accelerate_6_abs_max',
 'mfcc_1_quantile75',
 'q05_5',
 'abs_q95_9',
 'q05_roll_mean_100',
 'mfcc_10_abs_q25',
 'q75_9',
 'mfcc_15_mean',
 "change_quantiles{'ql': 0.6, 'qh': 0.8, 'isabs': True, 'f_agg': 'mean'}",
 'flac3_0_abs_std',
 "change_quantiles{'ql': 0.4, 'qh': 1.0, 'isabs': False, 'f_agg': 'mean'}"]

In [24]:
param822 = {'columns': top200,
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler', 'init':{}},
 'algorithm': {'cls': 'lgb.LGBMRegressor',
  'init': {'learning_rate': 0.20562566891912665,
   'feature_fraction': 0.6442827431025538,
   'bagging_fraction': 0.802261802916953,
   'min_data_in_leaf': 704,
   'lambda_l1': 65.43072344708864,
   'lambda_l2': 62.4903783326351,
   'max_bin': 96,
   'num_leaves': 4,
   'random_state': 5943,
   'n_jobs': 16},
  'fit': {}},
 }

In [77]:
mytrial =[]
EP.select_features_(df_train, param822, mytrial, nfeats_best=10, nfeats_removed_per_try=1, key='average_permutation_weight', remark='rfe to 10 group3')

In [26]:
mytrial =[]
EP.select_features_(df_train, param822, mytrial, nfeats_best=20, nfeats_removed_per_try=10, key='average_model_weight', remark='rfe to 10 group3 model weight')

In [27]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [29]:
df_trial[df_trial['remark']=='rfe to 10 group3'][['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff', 'remark']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
200,2019-05-30 07:44:54.376348,200,1.833266,0.001247,2.114163,0.007348,0.280897,rfe to 10 group3
201,2019-05-30 07:46:21.760008,190,1.836945,0.001168,2.105274,0.006524,0.268329,rfe to 10 group3
202,2019-05-30 07:47:46.915482,180,1.843670,0.001261,2.090790,0.005840,0.247120,rfe to 10 group3
203,2019-05-30 07:49:03.180761,170,1.863349,0.001791,2.075974,0.009361,0.212624,rfe to 10 group3
204,2019-05-30 07:50:18.579707,160,1.868806,0.001823,2.054681,0.011789,0.185875,rfe to 10 group3
205,2019-05-30 07:51:31.811278,150,1.874641,0.002162,2.059701,0.012927,0.185060,rfe to 10 group3
206,2019-05-30 07:52:40.306392,140,1.876497,0.001987,2.055771,0.011990,0.179274,rfe to 10 group3
207,2019-05-30 07:53:44.131969,130,1.878320,0.002075,2.045999,0.012080,0.167680,rfe to 10 group3
208,2019-05-30 07:54:46.528676,120,1.880426,0.002166,2.047209,0.012268,0.166783,rfe to 10 group3
209,2019-05-30 07:55:42.315509,110,1.886064,0.002286,2.046174,0.012501,0.160110,rfe to 10 group3


In [31]:
score = df_trial.loc[217].val_mae

In [32]:
param = df_trial.loc[217]['param']

In [40]:
def width_frist_rfe(df_train, param, trial, score, df_test=None, remark=None):

    param_ = copy.deepcopy(param)
    columns_ = param_['columns']
    best_score = score
    best_param = param_
    for col in columns_:
        param_['columns'] = list(set(columns_) - set([col]))
        df_his, df_feature_importances, df_valid_pred, df_test_pred = EP.process(df_train, param_, df_test=df_test, trial=trial, is_output_feature_importance=False, remark=remark)
        val_mae_mean = np.mean(df_his.valid)
        if val_mae_mean<best_score:
            best_score = val_mae_mean
            best_param = copy.deepcopy(param_)

    if best_score < score:
        width_frist_rfe(df_train, best_param, trial, best_score, df_test, remark=remark)

    return

In [41]:
mytrial=[]
width_frist_rfe(df_train, param, mytrial, score, df_test=df_test, remark='wf 217')

In [43]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [44]:
df_trial[(df_trial['remark']=='wf 217')].sort_values(by=['val_mae'])[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
615,2019-05-31 10:42:57.154150,26,1.924369,0.002107,2.038264,0.012776,0.113895
600,2019-05-31 10:38:04.907831,27,1.921733,0.002079,2.039145,0.014210,0.117412
616,2019-05-31 10:43:15.163221,26,1.922316,0.002172,2.039687,0.014872,0.117372
629,2019-05-31 10:49:13.308707,25,1.926718,0.002177,2.040725,0.012472,0.114006
621,2019-05-31 10:45:17.758022,26,1.922198,0.002096,2.040836,0.013688,0.118638
624,2019-05-31 10:46:47.740361,26,1.926255,0.002166,2.040850,0.012878,0.114595
642,2019-05-31 10:55:36.962151,25,1.923983,0.002091,2.041372,0.013562,0.117389
652,2019-05-31 11:00:34.663968,25,1.922588,0.002183,2.041373,0.013508,0.118785
635,2019-05-31 10:52:10.458742,25,1.924030,0.002238,2.041382,0.013515,0.117352
628,2019-05-31 10:48:44.072253,25,1.926393,0.002261,2.041794,0.012991,0.115401


In [45]:
param=df_trial.loc[615]['param']

In [46]:
param

{'columns': ['mfcc_10_abs_q75',
  'q25_roll_std_100',
  'iqr_6',
  'mfcc_9_mean',
  'abs_q75_7',
  'mfcc_delta_5_min',
  'mfcc_delta_3_quantile01',
  'abs_q75_6',
  'q05_roll_std_100',
  'mfcc_accelerate_1_kurtosis',
  'mfcc_5_mean',
  'spkt_welch_density__coeff_42',
  'mfcc_3_rolling_std_mean',
  'mfcc_12_mean',
  '5000smoothness_entropy_',
  'mfcc_13_mean',
  "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
  'mfcc_5_abs_max',
  'abs_q25_5',
  'mfcc_5_quantile95',
  'spkt_welch_densitycoeff_5',
  '5000crest_factor_quantile75',
  'spkt_welch_densitycoeff_2',
  'mfcc_accelerate_8_variance',
  "number_peaks{'n': 10}",
  'spkt_welch_density__coeff_3'],
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler', 'init': {}},
 'algorithm': {'cls': 'lgb.LGBMRegressor',
  'init': {'learning_rate': 0.20562566891912665,
   'feature_fraction': 0.6442827431025538,
   'bagging_fraction': 0.802261802916953,


In [49]:
mytrial = []
#  tune hypterparameters
def objective(trial):
        
    learning_rate = trial.suggest_uniform('learning_rate', .01, .5)
    feature_fraction = trial.suggest_uniform('feature_fraction', .6, 1)
    bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.6, 1)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 200, 800)
    lambda_l1 = trial.suggest_loguniform('lambda_l1', 1e-6, 1e2)
    lambda_l2 = trial.suggest_loguniform('lambda_l2', 1e-6, 1e2)
    max_bin = trial.suggest_int('max_bin', 10, 100)
    num_leaves = trial.suggest_int('num_leaves', 4, 64)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':df_trial.loc[615]['param']['columns'],
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'group'
        },
        'scaler':{
            'cls':'StandardScaler',
            'init':{}
        },
        'algorithm':{
            'cls':'lgb.LGBMRegressor',
            'init':{
                'learning_rate':learning_rate,
                'feature_fraction':feature_fraction,
                'bagging_fraction':bagging_fraction,
                'min_data_in_leaf':min_data_in_leaf,
                'lambda_l1':lambda_l1,
                'lambda_l2':lambda_l2,
                'max_bin':max_bin,
                'num_leaves':num_leaves,
                'random_state':random_state,
                'n_jobs':16
            },
            'fit':{
            },
        },
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 615')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

In [50]:
db.commit()